In [4]:
%pip install scipy --quiet
%pip install tenacity --quiet
%pip install tiktoken --quiet
%pip install termcolor --quiet
%pip install openai --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
import dotenv
import os

import os

# Try to get the GITHUB_TOKEN from the system environment variables
github_token = os.environ["GITHUB_KEY"]
if not github_token:
    raise ValueError("GITHUB_TOKEN is not set in your system environment variables")

# Set the required OpenAI environment variables using the retrieved token
os.environ["OPENAI_API_KEY"] = github_token
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

print("Environment variables have been set successfully.")

GPT_MODEL = "gpt-4o-mini"

client = OpenAI()

Environment variables have been set successfully.


In [12]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [16]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os

api = KaggleApi()
api.authenticate()

dataset = 'arevel/chess-games'  # Example: 'zillow/zecon'

api.dataset_download_files(dataset, path='.', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/arevel/chess-games


In [17]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chess_games.db')
cursor = conn.cursor()

df = pd.read_csv('chess_games.csv')

df.to_sql('Games', conn, if_exists='replace', index=False)

6256184

In [18]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [19]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [42]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What types of games have the most white wins?"
}]

response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message)

print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_v5ezBetGA8PB1P8lfczBUbT1', function=Function(arguments='{"query":"SELECT Opening, COUNT(Result) AS WhiteWins FROM Games WHERE Result = \'1-0\' GROUP BY Opening ORDER BY WhiteWins DESC LIMIT 10;"}', name='ask_database'), type='function')])


In [43]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = eval(tool_calls[0].function.arguments)['query']
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response_message.content) 

The following types of games have recorded the most white wins based on various openings:

1. **Scandinavian Defense: Mieses-Kotroc Variation** - 61,959 wins
2. **Van't Kruijs Opening** - 58,014 wins
3. **Modern Defense** - 51,785 wins
4. **Horwitz Defense** - 50,010 wins
5. **French Defense: Knight Variation** - 39,100 wins
6. **Owen Defense** - 38,087 wins
7. **Caro-Kann Defense** - 37,071 wins
8. **Scandinavian Defense** - 36,161 wins
9. **Sicilian Defense** - 35,958 wins
10. **Philidor Defense #3** - 35,697 wins

These openings have notably higher numbers of wins for White in games of chess.
